In [1]:
#!/usr/bin/env python3
import time
import rclpy
from rclpy.logging import get_logger
#import moveit python library
from moveit.core.robot_state import RobotState
from moveit.planning import (MoveItPy,MultiPipelinePlanRequestParameters)

import numpy as np
from geometry_msgs.msg import PoseStamped, Pose    # set pose goal with PoseStamped messsage

from math import pi

In [2]:
def plan_and_execute(
    robot,
    planning_component,
    logger,
    single_plan_parameters=None,
    multi_plan_parameters=None,
    sleep_time=0.0,
):
    """Helper function to plan and execute a motion."""
    # plan to goal
    logger.info("Planning trajectory")
    if multi_plan_parameters is not None:
        plan_result = planning_component.plan(
            multi_plan_parameters=multi_plan_parameters
        )
    elif single_plan_parameters is not None:
        plan_result = planning_component.plan(
            single_plan_parameters=single_plan_parameters
        )
    else:
        plan_result = planning_component.plan()

    # execute the plan
    if plan_result:
        logger.info("Executing plan")
        robot_trajectory = plan_result.trajectory
        robot.execute(robot_trajectory, controllers=[])
    else:
        logger.error("Planning failed")

    time.sleep(sleep_time)


In [3]:
def get_quaternion_from_euler(roll, pitch, yaw):
  """
  Convert an Euler angle to a quaternion.
   
  Input
    :param roll: The roll (rotation around x-axis) angle in radians.
    :param pitch: The pitch (rotation around y-axis) angle in radians.
    :param yaw: The yaw (rotation around z-axis) angle in radians.
 
  Output
    :return qx, qy, qz, qw: The orientation in quaternion [x,y,z,w] format
  """
  qx = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
  qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
  qz = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
  qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
 
  return [qx, qy, qz, qw]

In [4]:
def set_goal_from_angles(roll, pitch, yaw): # ATTENZIONE! " I would suggest using something like numpy-quaternion for making sure your input quaternion is valid and properly normalized, since it's very easy to run into precision issues"
    # Se i valori delle pose non sono ben normalizzati, il comando non diventa valido!
    # set start state to current state
    lite6_arm.set_start_state_to_current_state()

    quaternion = get_quaternion_from_euler (roll, pitch, yaw)

    pose_goal = Pose()
    #pose_goal.header.frame_id = "link_base"
    
    pose_goal.orientation.x = quaternion[0]
    pose_goal.orientation.y = quaternion[1]
    pose_goal.orientation.z = quaternion[2]
    pose_goal.orientation.w = quaternion[3]
    

    """
    lite6_arm.set_goal_state(pose_stamped_msg=pose_goal, pose_link="link_tcp")
    # plan to goal
    plan_and_execute(lite6, lite6_arm, logger, sleep_time=3.0)
    if failed == False:
        print("It has worked")
        # if the the configuration is valid, we store it in the list
        good_configurations.append([roll, pitch, yaw])
    else:
        print("NOT worked")
    """

In [23]:
# we need to specify our moveit_py config at the top of each notebook we use. 
# this is since we will start spinning a moveit_py node within this notebook.
from moveit_configs_utils import MoveItConfigsBuilder
from ament_index_python.packages import get_package_share_directory

moveit_config = (
        MoveItConfigsBuilder(robot_name="UF_ROBOT", package_name="lite6_enrico")
        .robot_description_semantic(file_path="config/UF_ROBOT.srdf")
        .trajectory_execution(file_path="config/moveit_controllers.yaml")
        .robot_description(file_path="config/UF_ROBOT.urdf.xacro")
        .robot_description_kinematics(file_path="config/kinematics.yaml")
        .joint_limits(file_path="config/joint_limits.yaml")
        .moveit_cpp(
            file_path=get_package_share_directory("lite6_moveit_demos")
            + "/config/moveit_cpp.yaml"
        )
        .to_moveit_configs()
    ).to_dict()

In [6]:
# initialise rclpy (only for logging purposes)
#rclpy.init()
logger = get_logger("moveit_py.pose_goal")

# instantiate moveit_py instance and a planning component for the panda_arm
lite6 = MoveItPy(node_name="moveit_py", config_dict=moveit_config)
lite6_arm = lite6.get_planning_component("lite6_arm")

[INFO] [1714386128.937665624] [moveit_3454781494.moveit.py.cpp_initializer]: Initialize rclcpp
[INFO] [1714386128.937747732] [moveit_3454781494.moveit.py.cpp_initializer]: Initialize node parameters
[INFO] [1714386128.937765743] [moveit_3454781494.moveit.py.cpp_initializer]: Initialize node and executor
[INFO] [1714386128.981647934] [moveit_3454781494.moveit.py.cpp_initializer]: Spin separate thread
[INFO] [1714386128.991378066] [moveit_3454781494.moveit.ros.rdf_loader]: Loaded robot model in 0.00951448 seconds
[INFO] [1714386128.991446956] [moveit_3454781494.moveit.core.robot_model]: Loading robot model 'UF_ROBOT'...
[INFO] [1714386128.991462222] [moveit_3454781494.moveit.core.robot_model]: No root/virtual joint specified in SRDF. Assuming fixed joint
[WARN] [1714386129.038166818] [moveit_3454781494.moveit.core.robot_model]: Could not identify parent group for end-effector 'link_tcp'
[INFO] [1714386129.060126296] [moveit_3454781494.moveit.kinematics.kdl_kinematics_plugin]: Joint weigh

In [ ]:
# KERNEL CRASHA, PERCHÉ? A quanto pare il kernel crasha quando si cerca di settare una posa iniziale e finale con lo stesso stato del robot. Perché?
# Il problema risiede nel fatto che il robot_state.update() non viene chiamato dopo aver settato la posa iniziale, quindi il kernel crasha perché non riesce a trovare una soluzione valida.

# FUNZIONA.

import random

for i in range(30):
    planning_scene_monitor = lite6.get_planning_scene_monitor()
    with planning_scene_monitor.read_write() as scene:
            robot_state = scene.current_state
            original_joint_positions = robot_state.get_joint_group_positions("lite6_arm")

            lite6_arm.set_start_state_to_current_state()

            check_init_pose = robot_state.get_pose("link_tcp")
            print("Initial_pose:", check_init_pose)












            #Set the pose goal, funziona sempre
            pose_goal = Pose()
            pose_goal.position.x = random.uniform(0.3, 0.4) + 0.01*i
            pose_goal.position.y = random.uniform(0.3, 0.4)
            pose_goal.position.z = random.uniform(0.2, 0.4)
            pose_goal.orientation.w = 1.0















            # Set the robot state and check collisions
            result = robot_state.set_from_ik("lite6_arm", pose_goal, "link_tcp", timeout=2.0)
            if not result:
                logger.error("IK solution was not found!")
            else:
                logger.info("IK solution found!")
            
            lite6_arm.set_goal_state(robot_state=robot_state)
            
            robot_state.update() # otherwise the kernel crashes

            check_updated_pose = robot_state.get_pose("link_tcp")

            print("New_pose:", check_updated_pose)

            logger.info("Go to goal")

            robot_state.set_joint_group_positions(
            "lite6_arm",
            original_joint_positions,
                )
            robot_state.update()

    plan_and_execute(lite6, lite6_arm, logger,sleep_time=1.0) 

In [60]:
from geometry_msgs.msg import PoseStamped, Pose    # set pose goal with PoseStamped messsage
def go_to_pose(roll, pitch, yaw):
        planning_scene_monitor = lite6.get_planning_scene_monitor()
        with planning_scene_monitor.read_write() as scene:

            # instantiate a RobotState instance using the current robot model
            robot_state = scene.current_state
            original_joint_positions = robot_state.get_joint_group_positions("lite6_arm")

            lite6_arm.set_start_state_to_current_state()

            check_init_pose = robot_state.get_pose("link_tcp")
            print("Initial_pose:", check_init_pose)

            #roll = -pi/2
            #pitch = random.uniform(0, 1)
            #yaw = random.uniform(0, 1)

            print("roll is", roll,"pitch is", pitch,"yaw is", yaw)
            print("Quaternions are", get_quaternion_from_euler(roll, pitch, yaw))
            
            #set_goal_from_angles(roll,pitch,yaw)


            quaternion = get_quaternion_from_euler (roll, pitch, yaw)

            pose_goal = Pose()
            #pose_goal.header.frame_id = "link_base"


            pose_goal.position.x = check_init_pose.position.x
            pose_goal.position.y = check_init_pose.position.y
            pose_goal.position.z = check_init_pose.position.z +0.001
            pose_goal.orientation.x = quaternion[0]
            pose_goal.orientation.y = quaternion[1]
            pose_goal.orientation.z = quaternion[2]
            pose_goal.orientation.w = quaternion[3]


            # Set the robot state and check collisions
            result = robot_state.set_from_ik("lite6_arm", pose_goal, "link_tcp", timeout=2.0)
            if not result:
                logger.error("IK solution was not found!")
                return
            else:
                logger.info("IK solution found!")
            
            lite6_arm.set_goal_state(robot_state=robot_state)
            
            robot_state.update() # otherwise the kernel crashes

            check_updated_pose = robot_state.get_pose("link_tcp")

            print("New_pose:", check_updated_pose)

            logger.info("Go to goal")

            robot_state.set_joint_group_positions(
            "lite6_arm",
            original_joint_positions,
                )
            robot_state.update()

        plan_and_execute(lite6, lite6_arm, logger,sleep_time=3.0) 

In [71]:
go_to_pose(-2*pi,-3*pi/2,0)

Initial_pose: geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.2117463777198852, y=0.1064182079513225, z=0.5010041083341993), orientation=geometry_msgs.msg.Quaternion(x=-0.5000000434160566, y=0.4999997652428239, z=0.4999999607596112, w=0.500000230581398))
roll is -6.283185307179586 pitch is -4.71238898038469 yaw is 0
Quaternions are [8.659560562354932e-17, 0.7071067811865476, -8.659560562354934e-17, 0.7071067811865475]
New_pose: geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.21174637752821943, y=0.10641820776790135, z=0.5020041083123605), orientation=geometry_msgs.msg.Quaternion(x=3.922186905392419e-11, y=0.707106781158807, z=1.6202262853458105e-11, w=0.7071067812142885))


[INFO] [1714387470.523764861] [moveit_py.pose_goal]: IK solution found!
[INFO] [1714387470.531160498] [moveit_py.pose_goal]: Go to goal
[INFO] [1714387470.543953900] [moveit_py.pose_goal]: Planning trajectory
[WARN] [1714387470.544034068] [moveit_py]: Parameter 'plan_request_params.planning_time' not found in config use default value instead, check parameter type and namespace in YAML file
[INFO] [1714387470.544228612] [moveit_py]: Calling PlanningRequestAdapter 'ResolveConstraintFrames'
[INFO] [1714387470.544353912] [moveit_py]: Calling PlanningRequestAdapter 'ValidateWorkspaceBounds'
[WARN] [1714387470.544372125] [moveit_3454781494.moveit.ros.validate_workspace_bounds]: It looks like the planning volume was not specified. Using default values.
[INFO] [1714387470.544440607] [moveit_py]: Calling PlanningRequestAdapter 'CheckStartStateBounds'
[INFO] [1714387470.544526244] [moveit_py]: Calling PlanningRequestAdapter 'CheckStartStateCollision'
[INFO] [1714387470.544700112] [moveit_3454781

**Il prossimo passo è quello di implementare matrici di rotazione per il movimento dell'EE del robot